## Chapter 25, Preprocessing and Feature Engineering

Code in the book and code in github repository differ quite a lot, especially at the beginning.

**Note:** I needed to change file names because I am loading them from community cloud DataBricks account. The files are the same as the ones provided in the STDG github.

In [3]:
sales = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/definitive-guide/data/retail-data/by-day/*.csv")\
  .coalesce(5)\
  .where("Description IS NOT NULL")
fakeIntDF = spark.read.parquet("/databricks-datasets/definitive-guide/data/simple-ml-integers")
simpleDF = spark.read.json("/databricks-datasets/definitive-guide/data/simple-ml")
scaleDF = spark.read.parquet("/databricks-datasets/definitive-guide/data/simple-ml-scaling")

In [4]:
sales.cache()
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom|
 580538| 23077| DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22906|12 MESSAGE CARDS ...| 24|2011-12-05 08:38:00| 1.65| 14075.0|United Kingdom|
 580538| 21914|BLUE HARMONICA IN...| 24|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22467| GUMBALL COAT RACK| 6|2011-12-05 08:38:00| 2.55| 14075.0|United Kingdom|
 580538| 21544|SKULLS WATER TRA...| 48|2011-12-05 08:38:00| 0.85| 14075.0|United Kingdom|
 580538| 23126|FELTCRAFT GIRL AM...| 8|2011-12-05 08:38:00| 4.95| 14075.0|United Kingdom|
 580538| 21833|CAMOUFLAGE LED TORCH| 24|2011-12-05 08:38:00| 1.69| 14075.0|United Kingdom|
 580539| 21479|WHITE SKULL HOT W...| 4|2011-12-05 08:39:00| 4.25| 18180.0|United Kingdom|
 580539| 84030E|ENGLISH ROSE HOT ...| 4|2011-12-05 08:39:00| 4.25| 18180.0|United Kingdom|
 580539| 23355|HOT WATER BOTTLE ...| 4|2011-12-05 08:39:00| 4.95| 18180.0|United Kingdom|
 580539| 22111|SCOTTIE DOG HOT W...| 3|2011-12-05 08:39:00| 4.95| 18180.0|United Kingdom|
 580539| 21115|ROSE CARAVAN DOOR...| 8|2011-12-05 08:39:00| 1.95| 18180.0|United Kingdom|
 580539| 21411|GINGHAM HEART DO...| 8|2011-12-05 08:39:00| 1.95| 18180.0|United Kingdom|
 580539| 23235|STORAGE TIN VINTA...| 12|2011-12-05 08:39:00| 1.25| 18180.0|United Kingdom|
 580539| 23239|SET OF 4 KNICK KN...| 6|2011-12-05 08:39:00| 1.65| 18180.0|United Kingdom|
 580539| 22197| POPCORN HOLDER| 36|2011-12-05 08:39:00| 0.85| 18180.0|United Kingdom|
 580539| 22693|GROW A FLYTRAP OR...| 24|2011-12-05 08:39:00| 1.25| 18180.0|United Kingdom|
 580539| 22372|AIRLINE BAG VINTA...| 4|2011-12-05 08:39:00| 4.25| 18180.0|United Kingdom|
 580539| 22375|AIRLINE BAG VINTA...| 4|2011-12-05 08:39:00| 4.25| 18180.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 20 rows

In [5]:
from pyspark.ml.feature import Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
Description |DescOut |
+-----------------------------------+------------------------------------------+
RABBIT NIGHT LIGHT |[rabbit, night, light] |
DOUGHNUT LIP GLOSS |[doughnut, lip, gloss] |
12 MESSAGE CARDS WITH ENVELOPES |[12, message, cards, with, envelopes] |
BLUE HARMONICA IN BOX |[blue, harmonica, in, box] |
GUMBALL COAT RACK |[gumball, coat, rack] |
SKULLS WATER TRANSFER TATTOOS |[skulls, , water, transfer, tattoos] |
FELTCRAFT GIRL AMELIE KIT |[feltcraft, girl, amelie, kit] |
CAMOUFLAGE LED TORCH |[camouflage, led, torch] |
WHITE SKULL HOT WATER BOTTLE |[white, skull, hot, water, bottle] |
ENGLISH ROSE HOT WATER BOTTLE |[english, rose, hot, water, bottle] |
HOT WATER BOTTLE KEEP CALM |[hot, water, bottle, keep, calm] |
SCOTTIE DOG HOT WATER BOTTLE |[scottie, dog, hot, water, bottle] |
ROSE CARAVAN DOORSTOP |[rose, caravan, doorstop] |
GINGHAM HEART DOORSTOP RED |[gingham, heart, , doorstop, red] |
STORAGE TIN VINTAGE LEAF |[storage, tin, vintage, leaf] |
SET OF 4 KNICK KNACK TINS POPPIES |[set, of, 4, knick, knack, tins, poppies] |
POPCORN HOLDER |[popcorn, holder] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[grow, a, flytrap, or, sunflower, in, tin]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[airline, bag, vintage, world, champion] |
AIRLINE BAG VINTAGE JET SET BROWN |[airline, bag, vintage, jet, set, brown] |
+-----------------------------------+------------------------------------------+
only showing top 20 rows

In [6]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+-----------------------------------+
 id| features|StandardScaler_d925f6abee94__output|
+---+--------------+-----------------------------------+
 0|[1.0,0.1,-1.0]| [1.19522860933439...|
 1| [2.0,1.1,1.0]| [2.39045721866878...|
 0|[1.0,0.1,-1.0]| [1.19522860933439...|
 1| [2.0,1.1,1.0]| [2.39045721866878...|
 1|[3.0,10.1,3.0]| [3.58568582800318...|
+---+--------------+-----------------------------------+

In [7]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------------------+-----+
color| lab|value1| value2| features|label|
+-----+----+------+------------------+--------------------+-----+
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 16|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red|good| 45| 38.97187133755819|(10,[0,2,3,4,7],[...| 1.0|
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
+-----+----+------+------------------+--------------------+-----+
only showing top 20 rows

In [8]:
from pyspark.ml.feature import SQLTransformer

basicTransformation = SQLTransformer()\
  .setStatement("""
    SELECT sum(Quantity), count(*), CustomerID
    FROM __THIS__
    GROUP BY CustomerID
  """)

basicTransformation.transform(sales).show()

+-------------+--------+----------+
sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
 119| 62| 14452.0|
 440| 143| 16916.0|
 630| 72| 17633.0|
 34| 6| 14768.0|
 1542| 30| 13094.0|
 854| 117| 17884.0|
 97| 12| 16596.0|
 290| 98| 13607.0|
 541| 27| 14285.0|
 244| 31| 16561.0|
 491| 152| 13956.0|
 204| 76| 13533.0|
 493| 64| 16629.0|
 159| 38| 17267.0|
 1140| 30| 13918.0|
 55| 28| 18114.0|
 88| 7| 14473.0|
 150| 16| 14024.0|
 206| 23| 12493.0|
 138| 18| 15776.0|
+-------------+--------+----------+
only showing top 20 rows

In [9]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

+----+----+----+------------------------------------+
int1|int2|int3|VectorAssembler_0e40691e802a__output|
+----+----+----+------------------------------------+
 1| 2| 3| [1.0,2.0,3.0]|
 4| 5| 6| [4.0,5.0,6.0]|
 7| 8| 9| [7.0,8.0,9.0]|
+----+----+----+------------------------------------+

In [10]:
contDF = spark.range(20).selectExpr("cast(id as double)")


In [11]:
from pyspark.ml.feature import Bucketizer
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
 id|Bucketizer_65ca7107f3dd__output|
+----+-------------------------------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 0.0|
 4.0| 0.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 1.0|
 8.0| 1.0|
 9.0| 1.0|
10.0| 2.0|
11.0| 2.0|
12.0| 2.0|
13.0| 2.0|
14.0| 2.0|
15.0| 2.0|
16.0| 2.0|
17.0| 2.0|
18.0| 2.0|
19.0| 2.0|
+----+-------------------------------+

You may note that our transformations are done in a similar manner: we import an object with transformation methods, and apply `.transform` method to our data. Although the command looks exactly the same but when utilized with different object it works differently. With `Tokenizer` object the command separates a record description into singe words, and with `Bucketizer` object we get a bucket number for each row. Below we will see that this method is preceeded by additional one, `.fit`. It is here because this time we need additional information from our data for the transformation to work: cutoff points for quantiles. The `.fit` method extracts the information. In the STDG such method is called a Spark estimator.

In [13]:
from pyspark.ml.feature import QuantileDiscretizer
discretizer = QuantileDiscretizer(numBuckets=5, inputCol="id", outputCol ='quantiles')
result = discretizer.fit(contDF).transform(contDF)
result.show()

+----+---------+
 id|quantiles|
+----+---------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 1.0|
 4.0| 1.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 2.0|
 8.0| 2.0|
 9.0| 2.0|
10.0| 2.0|
11.0| 2.0|
12.0| 3.0|
13.0| 3.0|
14.0| 3.0|
15.0| 4.0|
16.0| 4.0|
17.0| 4.0|
18.0| 4.0|
19.0| 4.0|
+----+---------+

For `StandardScaler` object we need additional information, too: a column mean and its standard deviation. 

These transformations are done in 2 stages because we often need to apply the same transformation to our test set. For this our transformation object keeps the information which was derived from our data, and we can apply a required transformation with `.transform` method only. When we work with text data such approach allows to ignore new words in test set which were not in our train set. 

By the way many tranfsormation objects have `.inverse_transform` method as well. Because if you transformed your output (`label`), for example, binarizing, then your predictions will be in the same format, and you will want them in the original form.

In [15]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+-----------------------------------+
 id| features|StandardScaler_9d6525ca1a25__output|
+---+--------------+-----------------------------------+
 0|[1.0,0.1,-1.0]| [1.19522860933439...|
 1| [2.0,1.1,1.0]| [2.39045721866878...|
 0|[1.0,0.1,-1.0]| [1.19522860933439...|
 1| [2.0,1.1,1.0]| [2.39045721866878...|
 1|[3.0,10.1,3.0]| [3.58568582800318...|
+---+--------------+-----------------------------------+

In [16]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------------------+
 id| features|MinMaxScaler_3eccf3759829__output|
+---+--------------+---------------------------------+
 0|[1.0,0.1,-1.0]| [5.0,5.0,5.0]|
 1| [2.0,1.1,1.0]| [7.5,5.5,7.5]|
 0|[1.0,0.1,-1.0]| [5.0,5.0,5.0]|
 1| [2.0,1.1,1.0]| [7.5,5.5,7.5]|
 1|[3.0,10.1,3.0]| [10.0,10.0,10.0]|
+---+--------------+---------------------------------+

In [17]:
from pyspark.ml.feature import MaxAbsScaler
maScaler = MaxAbsScaler().setInputCol("features")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show()

+---+--------------+---------------------------------+
 id| features|MaxAbsScaler_3d3661efba93__output|
+---+--------------+---------------------------------+
 0|[1.0,0.1,-1.0]| [0.33333333333333...|
 1| [2.0,1.1,1.0]| [0.66666666666666...|
 0|[1.0,0.1,-1.0]| [0.33333333333333...|
 1| [2.0,1.1,1.0]| [0.66666666666666...|
 1|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+---+--------------+---------------------------------+

In [18]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct()\
  .setScalingVec(scaleUpVec)\
  .setInputCol("features")
scalingUp.transform(scaleDF).show()

+---+--------------+---------------------------------------+
 id| features|ElementwiseProduct_89bc346080ea__output|
+---+--------------+---------------------------------------+
 0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
 1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
 0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
 1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
 1|[3.0,10.1,3.0]| [30.0,151.5,60.0]|
+---+--------------+---------------------------------------+

In [19]:
from pyspark.ml.feature import Normalizer
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show()

+---+--------------+-------------------------------+
 id| features|Normalizer_0052afa86191__output|
+---+--------------+-------------------------------+
 0|[1.0,0.1,-1.0]| [0.47619047619047...|
 1| [2.0,1.1,1.0]| [0.48780487804878...|
 0|[1.0,0.1,-1.0]| [0.47619047619047...|
 1| [2.0,1.1,1.0]| [0.48780487804878...|
 1|[3.0,10.1,3.0]| [0.18633540372670...|
+---+--------------+-------------------------------+

In [20]:
from pyspark.ml.feature import StringIndexer
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()

+-----+----+------+------------------+--------+
color| lab|value1| value2|labelInd|
+-----+----+------+------------------+--------+
green|good| 1|14.386294994851129| 1.0|
 blue| bad| 8|14.386294994851129| 0.0|
 blue| bad| 12|14.386294994851129| 0.0|
green|good| 15| 38.97187133755819| 1.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 0.0|
 red|good| 35|14.386294994851129| 1.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 0.0|
 red| bad| 16|14.386294994851129| 0.0|
 red|good| 45| 38.97187133755819| 1.0|
green|good| 1|14.386294994851129| 1.0|
 blue| bad| 8|14.386294994851129| 0.0|
 blue| bad| 12|14.386294994851129| 0.0|
green|good| 15| 38.97187133755819| 1.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 0.0|
 red|good| 35|14.386294994851129| 1.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 0.0|
+-----+----+------+------------------+--------+
only showing top 20 rows

In [21]:
valIndexer = StringIndexer().setInputCol("value1").setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show(5)

+-----+----+------+------------------+--------+
color| lab|value1| value2|valueInd|
+-----+----+------+------------------+--------+
green|good| 1|14.386294994851129| 2.0|
 blue| bad| 8|14.386294994851129| 4.0|
 blue| bad| 12|14.386294994851129| 0.0|
green|good| 15| 38.97187133755819| 5.0|
green|good| 12|14.386294994851129| 0.0|
+-----+----+------+------------------+--------+
only showing top 5 rows

In [22]:
from pyspark.ml.feature import IndexToString
labelReverse = IndexToString().setInputCol("labelInd")
labelReverse.transform(idxRes).show(5)

+-----+----+------+------------------+--------+----------------------------------+
color| lab|value1| value2|labelInd|IndexToString_0b2c36cb3fab__output|
+-----+----+------+------------------+--------+----------------------------------+
green|good| 1|14.386294994851129| 1.0| good|
 blue| bad| 8|14.386294994851129| 0.0| bad|
 blue| bad| 12|14.386294994851129| 0.0| bad|
green|good| 15| 38.97187133755819| 1.0| good|
green|good| 12|14.386294994851129| 1.0| good|
+-----+----+------+------------------+--------+----------------------------------+
only showing top 5 rows

In [23]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
idxIn = spark.createDataFrame([
  (Vectors.dense(1, 2, 3),1),
  (Vectors.dense(2, 5, 6),2),
  (Vectors.dense(1, 8, 9),3)
]).toDF("features", "label")
indxr = VectorIndexer()\
  .setInputCol("features")\
  .setOutputCol("idxed")\
  .setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show()

+-------------+-----+-------------+
 features|label| idxed|
+-------------+-----+-------------+
[1.0,2.0,3.0]| 1|[0.0,2.0,3.0]|
[2.0,5.0,6.0]| 2|[1.0,5.0,6.0]|
[1.0,8.0,9.0]| 3|[0.0,8.0,9.0]|
+-------------+-----+-------------+

In [24]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd")
ohe.transform(colorLab).show(10)

+-----+--------+----------------------------------+
color|colorInd|OneHotEncoder_c97639b13858__output|
+-----+--------+----------------------------------+
green| 1.0| (2,[1],[1.0])|
 blue| 2.0| (2,[],[])|
 blue| 2.0| (2,[],[])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
+-----+--------+----------------------------------+
only showing top 10 rows

The following cell was `RegexTokenizer` instead of `Tokenizer` as it was presented in the book.

In [26]:
from pyspark.ml.feature import Tokenizer
tkn = Tokenizer()\
  .setInputCol("Description")\
  .setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
Description |DescOut |
+-----------------------------------+------------------------------------------+
RABBIT NIGHT LIGHT |[rabbit, night, light] |
DOUGHNUT LIP GLOSS |[doughnut, lip, gloss] |
12 MESSAGE CARDS WITH ENVELOPES |[12, message, cards, with, envelopes] |
BLUE HARMONICA IN BOX |[blue, harmonica, in, box] |
GUMBALL COAT RACK |[gumball, coat, rack] |
SKULLS WATER TRANSFER TATTOOS |[skulls, , water, transfer, tattoos] |
FELTCRAFT GIRL AMELIE KIT |[feltcraft, girl, amelie, kit] |
CAMOUFLAGE LED TORCH |[camouflage, led, torch] |
WHITE SKULL HOT WATER BOTTLE |[white, skull, hot, water, bottle] |
ENGLISH ROSE HOT WATER BOTTLE |[english, rose, hot, water, bottle] |
HOT WATER BOTTLE KEEP CALM |[hot, water, bottle, keep, calm] |
SCOTTIE DOG HOT WATER BOTTLE |[scottie, dog, hot, water, bottle] |
ROSE CARAVAN DOORSTOP |[rose, caravan, doorstop] |
GINGHAM HEART DOORSTOP RED |[gingham, heart, , doorstop, red] |
STORAGE TIN VINTAGE LEAF |[storage, tin, vintage, leaf] |
SET OF 4 KNICK KNACK TINS POPPIES |[set, of, 4, knick, knack, tins, poppies] |
POPCORN HOLDER |[popcorn, holder] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[grow, a, flytrap, or, sunflower, in, tin]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[airline, bag, vintage, world, champion] |
AIRLINE BAG VINTAGE JET SET BROWN |[airline, bag, vintage, jet, set, brown] |
+-----------------------------------+------------------------------------------+
only showing top 20 rows

In [27]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
  .setInputCol("Description")\
  .setOutputCol("DescOut")\
  .setPattern(" ")\
  .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------------------------------+
Description |DescOut |
+-----------------------------------+------------------------------------------+
RABBIT NIGHT LIGHT |[rabbit, night, light] |
DOUGHNUT LIP GLOSS |[doughnut, lip, gloss] |
12 MESSAGE CARDS WITH ENVELOPES |[12, message, cards, with, envelopes] |
BLUE HARMONICA IN BOX |[blue, harmonica, in, box] |
GUMBALL COAT RACK |[gumball, coat, rack] |
SKULLS WATER TRANSFER TATTOOS |[skulls, water, transfer, tattoos] |
FELTCRAFT GIRL AMELIE KIT |[feltcraft, girl, amelie, kit] |
CAMOUFLAGE LED TORCH |[camouflage, led, torch] |
WHITE SKULL HOT WATER BOTTLE |[white, skull, hot, water, bottle] |
ENGLISH ROSE HOT WATER BOTTLE |[english, rose, hot, water, bottle] |
HOT WATER BOTTLE KEEP CALM |[hot, water, bottle, keep, calm] |
SCOTTIE DOG HOT WATER BOTTLE |[scottie, dog, hot, water, bottle] |
ROSE CARAVAN DOORSTOP |[rose, caravan, doorstop] |
GINGHAM HEART DOORSTOP RED |[gingham, heart, doorstop, red] |
STORAGE TIN VINTAGE LEAF |[storage, tin, vintage, leaf] |
SET OF 4 KNICK KNACK TINS POPPIES |[set, of, 4, knick, knack, tins, poppies] |
POPCORN HOLDER |[popcorn, holder] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[grow, a, flytrap, or, sunflower, in, tin]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[airline, bag, vintage, world, champion] |
AIRLINE BAG VINTAGE JET SET BROWN |[airline, bag, vintage, jet, set, brown] |
+-----------------------------------+------------------------------------------+
only showing top 20 rows

We see a lot of spaces in the left column, as we are supposed.

In [29]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
  .setInputCol("Description")\
  .setOutputCol("DescOut")\
  .setPattern(" ")\
  .setGaps(False)\
  .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------+
Description |DescOut |
+-----------------------------------+------------------+
RABBIT NIGHT LIGHT |[ , ] |
DOUGHNUT LIP GLOSS |[ , , ] |
12 MESSAGE CARDS WITH ENVELOPES |[ , , , ] |
BLUE HARMONICA IN BOX |[ , , , ] |
GUMBALL COAT RACK |[ , ] |
SKULLS WATER TRANSFER TATTOOS |[ , , , , ] |
FELTCRAFT GIRL AMELIE KIT |[ , , ] |
CAMOUFLAGE LED TORCH |[ , ] |
WHITE SKULL HOT WATER BOTTLE |[ , , , , ] |
ENGLISH ROSE HOT WATER BOTTLE |[ , , , ] |
HOT WATER BOTTLE KEEP CALM |[ , , , ] |
SCOTTIE DOG HOT WATER BOTTLE |[ , , , ] |
ROSE CARAVAN DOORSTOP |[ , ] |
GINGHAM HEART DOORSTOP RED |[ , , , ] |
STORAGE TIN VINTAGE LEAF |[ , , ] |
SET OF 4 KNICK KNACK TINS POPPIES |[ , , , , , ]|
POPCORN HOLDER |[ ] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[ , , , , , ]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[ , , , , ] |
AIRLINE BAG VINTAGE JET SET BROWN |[ , , , , ] |
+-----------------------------------+------------------+
only showing top 20 rows

In [30]:
from pyspark.ml.feature import StopWordsRemover
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
  .setStopWords(englishStopWords)\
  .setInputCol("DescOut")
stops.transform(tokenized).show()

+--------------------+--------------------+-------------------------------------+
 Description| DescOut|StopWordsRemover_b487699f1da6__output|
+--------------------+--------------------+-------------------------------------+
 RABBIT NIGHT LIGHT|[rabbit, night, l...| [rabbit, night, l...|
 DOUGHNUT LIP GLOSS |[doughnut, lip, g...| [doughnut, lip, g...|
12 MESSAGE CARDS ...|[12, message, car...| [12, message, car...|
BLUE HARMONICA IN...|[blue, harmonica,...| [blue, harmonica,...|
 GUMBALL COAT RACK|[gumball, coat, r...| [gumball, coat, r...|
SKULLS WATER TRA...|[skulls, , water,...| [skulls, , water,...|
FELTCRAFT GIRL AM...|[feltcraft, girl,...| [feltcraft, girl,...|
CAMOUFLAGE LED TORCH|[camouflage, led,...| [camouflage, led,...|
WHITE SKULL HOT W...|[white, skull, ho...| [white, skull, ho...|
ENGLISH ROSE HOT ...|[english, rose, h...| [english, rose, h...|
HOT WATER BOTTLE ...|[hot, water, bott...| [hot, water, bott...|
SCOTTIE DOG HOT W...|[scottie, dog, ho...| [scottie, dog, ho...|
ROSE CARAVAN DOOR...|[rose, caravan, d...| [rose, caravan, d...|
GINGHAM HEART DO...|[gingham, heart, ...| [gingham, heart, ...|
STORAGE TIN VINTA...|[storage, tin, vi...| [storage, tin, vi...|
SET OF 4 KNICK KN...|[set, of, 4, knic...| [set, 4, knick, k...|
 POPCORN HOLDER| [popcorn, holder]| [popcorn, holder]|
GROW A FLYTRAP OR...|[grow, a, flytrap...| [grow, flytrap, s...|
AIRLINE BAG VINTA...|[airline, bag, vi...| [airline, bag, vi...|
AIRLINE BAG VINTA...|[airline, bag, vi...| [airline, bag, vi...|
+--------------------+--------------------+-------------------------------------+
only showing top 20 rows

The `NGram` method did not work. Maybe it was depreciated. I looked up its documentation to fix it.

In [32]:
from pyspark.ml.feature import NGram
unigram = NGram(n=1, inputCol="DescOut", outputCol="unigrams")
unigramDataFrame = unigram.transform(tokenized)
unigramDataFrame.select("unigrams").show(truncate=False)

+------------------------------------------+
unigrams |
+------------------------------------------+
[rabbit, night, light] |
[doughnut, lip, gloss] |
[12, message, cards, with, envelopes] |
[blue, harmonica, in, box] |
[gumball, coat, rack] |
[skulls, , water, transfer, tattoos] |
[feltcraft, girl, amelie, kit] |
[camouflage, led, torch] |
[white, skull, hot, water, bottle] |
[english, rose, hot, water, bottle] |
[hot, water, bottle, keep, calm] |
[scottie, dog, hot, water, bottle] |
[rose, caravan, doorstop] |
[gingham, heart, , doorstop, red] |
[storage, tin, vintage, leaf] |
[set, of, 4, knick, knack, tins, poppies] |
[popcorn, holder] |
[grow, a, flytrap, or, sunflower, in, tin]|
[airline, bag, vintage, world, champion] |
[airline, bag, vintage, jet, set, brown] |
+------------------------------------------+
only showing top 20 rows

In [33]:
bigram = NGram(n=2, inputCol="DescOut", outputCol="bigrams")
bigramDataFrame = bigram.transform(tokenized)
bigramDataFrame.select("bigrams").show(truncate=False)

+-------------------------------------------------------------------+
bigrams |
+-------------------------------------------------------------------+
[rabbit night, night light] |
[doughnut lip, lip gloss] |
[12 message, message cards, cards with, with envelopes] |
[blue harmonica, harmonica in, in box] |
[gumball coat, coat rack] |
[skulls , water, water transfer, transfer tattoos] |
[feltcraft girl, girl amelie, amelie kit] |
[camouflage led, led torch] |
[white skull, skull hot, hot water, water bottle] |
[english rose, rose hot, hot water, water bottle] |
[hot water, water bottle, bottle keep, keep calm] |
[scottie dog, dog hot, hot water, water bottle] |
[rose caravan, caravan doorstop] |
[gingham heart, heart , doorstop, doorstop red] |
[storage tin, tin vintage, vintage leaf] |
[set of, of 4, 4 knick, knick knack, knack tins, tins poppies] |
[popcorn holder] |
[grow a, a flytrap, flytrap or, or sunflower, sunflower in, in tin]|
[airline bag, bag vintage, vintage world, world champion] |
[airline bag, bag vintage, vintage jet, jet set, set brown] |
+-------------------------------------------------------------------+
only showing top 20 rows

Here the `CountVectorizer.fit` makes a frequency table for all words, takes the first 500 the most frequent, and creates a table for the word frequencies for each row. This is a truly sparse table. All 500 words are kept in `CountVectorizer` object, so a similar transformation can be applied to a test set and the resulting table will have the same columns for the same words.

In [35]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol = "DescOut", outputCol = "countVec", vocabSize = 500, minDF=2.0, minTF =1.0)
fittedCV = cv.fit(tokenized)
result = fittedCV.transform(tokenized)
result.show()

+--------------------+--------------------+--------------------+
 Description| DescOut| countVec|
+--------------------+--------------------+--------------------+
 RABBIT NIGHT LIGHT|[rabbit, night, l...|(500,[150,185,212...|
 DOUGHNUT LIP GLOSS |[doughnut, lip, g...|(500,[462,463,492...|
12 MESSAGE CARDS ...|[12, message, car...|(500,[35,41,166],...|
BLUE HARMONICA IN...|[blue, harmonica,...|(500,[10,16,36,35...|
 GUMBALL COAT RACK|[gumball, coat, r...|(500,[228,280,408...|
SKULLS WATER TRA...|[skulls, , water,...|(500,[11,40,133],...|
FELTCRAFT GIRL AM...|[feltcraft, girl,...|(500,[60,64,69],[...|
CAMOUFLAGE LED TORCH|[camouflage, led,...| (500,[264],[1.0])|
WHITE SKULL HOT W...|[white, skull, ho...|(500,[15,34,39,40...|
ENGLISH ROSE HOT ...|[english, rose, h...|(500,[34,39,40,46...|
HOT WATER BOTTLE ...|[hot, water, bott...|(500,[34,39,40,14...|
SCOTTIE DOG HOT W...|[scottie, dog, ho...|(500,[34,39,40,14...|
ROSE CARAVAN DOOR...|[rose, caravan, d...|(500,[46,297],[1....|
GINGHAM HEART DO...|[gingham, heart, ...|(500,[3,4,11,143,...|
STORAGE TIN VINTA...|[storage, tin, vi...|(500,[6,45,109,16...|
SET OF 4 KNICK KN...|[set, of, 4, knic...|(500,[0,1,49,70,3...|
 POPCORN HOLDER| [popcorn, holder]|(500,[21,296],[1....|
GROW A FLYTRAP OR...|[grow, a, flytrap...|(500,[36,45,378],...|
AIRLINE BAG VINTA...|[airline, bag, vi...|(500,[2,6,328],[1...|
AIRLINE BAG VINTA...|[airline, bag, vi...|(500,[0,2,6,328,4...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [36]:
tfIdfIn = tokenized\
  .where("array_contains(DescOut, 'red')")\
  .select("DescOut")\
  .limit(10)
tfIdfIn.show(10, False)

+---------------------------------------+
DescOut |
+---------------------------------------+
[gingham, heart, , doorstop, red] |
[red, floral, feltcraft, shoulder, bag]|
[alarm, clock, bakelike, red] |
[pin, cushion, babushka, red] |
[red, retrospot, mini, cases] |
[red, kitchen, scales] |
[gingham, heart, , doorstop, red] |
[large, red, babushka, notebook] |
[red, retrospot, oven, glove] |
[red, retrospot, plate] |
+---------------------------------------+

In [37]:
from pyspark.ml.feature import HashingTF, IDF
tf = HashingTF()\
  .setInputCol("DescOut")\
  .setOutputCol("TFOut")\
  .setNumFeatures(10000)
idf = IDF()\
  .setInputCol("TFOut")\
  .setOutputCol("IDFOut")\
  .setMinDocFreq(2)

In [38]:
idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(10, False)

+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
DescOut |TFOut |IDFOut |
+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
[gingham, heart, , doorstop, red] |(10000,[3372,4291,4370,6594,9160],[1.0,1.0,1.0,1.0,1.0])|(10000,[3372,4291,4370,6594,9160],[1.2992829841302609,0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
[red, floral, feltcraft, shoulder, bag]|(10000,[155,1152,4291,5981,6756],[1.0,1.0,1.0,1.0,1.0]) |(10000,[155,1152,4291,5981,6756],[0.0,0.0,0.0,0.0,0.0]) |
[alarm, clock, bakelike, red] |(10000,[4291,4852,4995,9668],[1.0,1.0,1.0,1.0]) |(10000,[4291,4852,4995,9668],[0.0,0.0,0.0,0.0]) |
[pin, cushion, babushka, red] |(10000,[4291,5111,5673,7153],[1.0,1.0,1.0,1.0]) |(10000,[4291,5111,5673,7153],[0.0,0.0,0.0,1.2992829841302609]) |
[red, retrospot, mini, cases] |(10000,[547,1576,2591,4291],[1.0,1.0,1.0,1.0]) |(10000,[547,1576,2591,4291],[0.0,0.0,1.0116009116784799,0.0]) |
[red, kitchen, scales] |(10000,[3461,4291,6214],[1.0,1.0,1.0]) |(10000,[3461,4291,6214],[0.0,0.0,0.0]) |
[gingham, heart, , doorstop, red] |(10000,[3372,4291,4370,6594,9160],[1.0,1.0,1.0,1.0,1.0])|(10000,[3372,4291,4370,6594,9160],[1.2992829841302609,0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
[large, red, babushka, notebook] |(10000,[2782,2787,4291,7153],[1.0,1.0,1.0,1.0]) |(10000,[2782,2787,4291,7153],[0.0,0.0,0.0,1.2992829841302609]) |
[red, retrospot, oven, glove] |(10000,[302,2591,4291,8242],[1.0,1.0,1.0,1.0]) |(10000,[302,2591,4291,8242],[0.0,1.0116009116784799,0.0,0.0]) |
[red, retrospot, plate] |(10000,[2591,4291,4456],[1.0,1.0,1.0]) |(10000,[2591,4291,4456],[1.0116009116784799,0.0,0.0]) |
+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+

In [39]:
from pyspark.ml.feature import Word2Vec
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text",
  outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [0.000364033132792,-0.0473175618798,-0.0372836232185]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [-0.0449876114726,0.0245432928205,-0.04744969947]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.0543659523129,-0.000254406034946,-0.0141884714365]

In [40]:
from pyspark.ml.feature import PCA
pca = PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(20, False)

+---+--------------+------------------------------------------+
id |features |PCA_17af88c4c849__output |
+---+--------------+------------------------------------------+
0 |[1.0,0.1,-1.0]|[0.0713719499248418,-0.4526654888147822] |
1 |[2.0,1.1,1.0] |[-1.680494698407372,1.259340132221917] |
0 |[1.0,0.1,-1.0]|[0.0713719499248418,-0.4526654888147822] |
1 |[2.0,1.1,1.0] |[-1.680494698407372,1.259340132221917] |
1 |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060150646]|
+---+--------------+------------------------------------------+

In [41]:
from pyspark.ml.feature import PolynomialExpansion
pe = PolynomialExpansion().setInputCol("features").setDegree(2).setOutputCol("polyFeatures")
pe.transform(scaleDF).show()

+---+--------------+--------------------+
 id| features| polyFeatures|
+---+--------------+--------------------+
 0|[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,...|
 1| [2.0,1.1,1.0]|[2.0,4.0,1.1,2.2,...|
 0|[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,...|
 1| [2.0,1.1,1.0]|[2.0,4.0,1.1,2.2,...|
 1|[3.0,10.1,3.0]|[3.0,9.0,10.1,30....|
+---+--------------+--------------------+

In [42]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
  .transform(sales.select("Description", "CustomerId"))\
  .where("CustomerId IS NOT NULL")
prechi = fittedCV.transform(tokenized)\
  .where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
  .setFeaturesCol("countVec")\
  .setLabelCol("CustomerId")\
  .setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi)\
  .drop("customerId", "Description", "DescOut").show()

+--------------------+----------------------------------+
 countVec|ChiSqSelector_a518b9236476__output|
+--------------------+----------------------------------+
(500,[150,185,212...| (2,[],[])|
(500,[462,463,492...| (2,[],[])|
(500,[35,41,166],...| (2,[],[])|
(500,[10,16,36,35...| (2,[],[])|
(500,[228,280,408...| (2,[],[])|
(500,[11,40,133],...| (2,[],[])|
(500,[60,64,69],[...| (2,[],[])|
 (500,[264],[1.0])| (2,[],[])|
(500,[15,34,39,40...| (2,[],[])|
(500,[34,39,40,46...| (2,[],[])|
(500,[34,39,40,14...| (2,[],[])|
(500,[34,39,40,14...| (2,[],[])|
(500,[46,297],[1....| (2,[],[])|
(500,[3,4,11,143,...| (2,[],[])|
(500,[6,45,109,16...| (2,[],[])|
(500,[0,1,49,70,3...| (2,[0,1],[1.0,1.0])|
(500,[21,296],[1....| (2,[],[])|
(500,[36,45,378],...| (2,[],[])|
(500,[2,6,328],[1...| (2,[],[])|
(500,[0,2,6,328,4...| (2,[0],[1.0])|
+--------------------+----------------------------------+
only showing top 20 rows

In [43]:
fittedPCA = pca.fit(scaleDF)
fittedPCA.write().overwrite().save("/tmp/fittedPCA")

In [44]:
from pyspark.ml.feature import PCAModel
loadedPCA = PCAModel.load("/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

+---+--------------+------------------------+
 id| features|PCA_17af88c4c849__output|
+---+--------------+------------------------+
 0|[1.0,0.1,-1.0]| [0.07137194992484...|
 1| [2.0,1.1,1.0]| [-1.6804946984073...|
 0|[1.0,0.1,-1.0]| [0.07137194992484...|
 1| [2.0,1.1,1.0]| [-1.6804946984073...|
 1|[3.0,10.1,3.0]| [-10.872398139848...|
+---+--------------+------------------------+

At the end of the chapter we can see a custom transformer in Scala but not in Python.